In [20]:
#Import numpy so we can manipulate our rasters as multidimensional arrays
import numpy as np
import rasterio
import glob
import os

In [21]:
# File and folder paths
dirpath = r'/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/'


# Make a search criteria to select the DEM files
search_criteria = "*.tif"
q = os.path.join(dirpath, search_criteria)

files = glob.glob(q)

In [22]:
for file in files:
    with rasterio.open(file) as src:    
    # Read as numpy array
        array = src.read()
        profile = src.profile

        # Reclassify
        array[np.where(array <= 2000)] = 1
        array[np.where(array > 2000)] = -999999999

    with rasterio.open(file[:-4]+'_2kmBuffer.tif', 'w', **profile) as dst:
    # Write to disk
        dst.write(array)


#Now for merging everything together by taking the max pixel

In [26]:
# File and folder paths
dirpath = r'/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/'


# Make a search criteria to select the DEM files
search_criteria = "*2kmBuffer.tif"
q = os.path.join(dirpath, search_criteria)

files = glob.glob(q)
files

['/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/AfricaCenter_2kmBuffer.tif',
 '/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/AfricaEast_2kmBuffer.tif',
 '/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/AfricaNorth_2kmBuffer.tif',
 '/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/AfricaSouth_2kmBuffer.tif',
 '/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/AfricaWest_2kmBuffer.tif',
 '/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/AmericaCenter_2kmBuffer.tif',
 '/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/AsiaCenter_2kmBuffer.tif',
 '/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/AsiaNorth_2kmBuffer.tif',
 '/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/AsiaSouthEast_2kmBuffer.tif',
 '/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/AsiaSouth_2kmBuffer.tif',
 '/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclid

In [27]:
src_files_to_mosaic = []
for file in files:
    src = rasterio.open(file)
    src_files_to_mosaic.append(src)
src_files_to_mosaic

[<open DatasetReader name='/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/AfricaCenter_2kmBuffer.tif' mode='r'>,
 <open DatasetReader name='/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/AfricaEast_2kmBuffer.tif' mode='r'>,
 <open DatasetReader name='/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/AfricaNorth_2kmBuffer.tif' mode='r'>,
 <open DatasetReader name='/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/AfricaSouth_2kmBuffer.tif' mode='r'>,
 <open DatasetReader name='/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/AfricaWest_2kmBuffer.tif' mode='r'>,
 <open DatasetReader name='/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/AmericaCenter_2kmBuffer.tif' mode='r'>,
 <open DatasetReader name='/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/AsiaCenter_2kmBuffer.tif' mode='r'>,
 <open DatasetReader name='/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/AsiaNo

In [33]:
from rasterio.merge import merge
mosaic, out_trans = merge(src_files_to_mosaic, method='max')

In [ ]:
from rasterio.plot import show
show(mosaic, cmap='terrain')

In [34]:
src.crs

CRS.from_epsg(4326)

In [35]:
out_meta = src.meta.copy()
out_meta.update({"driver": "GTiff",
                 "height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_trans,
                }
               )

In [36]:
out_fp = r'/Volumes/GUILHERME/_microsoft_missing_from_OSM_roads/_euclidean/World_2kmBuffer.tif'

with rasterio.open(out_fp, "w", **out_meta) as dest:
    dest.write(mosaic)